In [85]:
import cv2
import numpy as np

MIN_HEIGHT = 100
MIN_WIDTH = 100
MIN_BRIGHTNESS = 50


def check_image_quality(image_path):
    try:
        image = cv2.imread(image_path)
        if image is None:
            print(f"Image not found or could not be loaded: {image_path}")
            return False


        if image.shape[0] < MIN_HEIGHT or image.shape[1] < MIN_WIDTH:
            return False

        # Example: check if the image is too dar
        if np.mean(image) < MIN_BRIGHTNESS:
            return False

        return True
    except Exception as e:
        print(f"Error checking image quality: {e}")
        return False

# Usage
is_good_quality = check_image_quality('data/William/images/William_04f57bf9-74b4-4269-adcc-6d7096345f86.png')
is_good_quality

True

In [86]:
def check_eye_region(image, eye_box):
    """
    Check if the eye region defined by the bounding box is valid.
    Args:
        image: The image in which the eye region is located.
        eye_box: The bounding box of the eye region in the format [x, y, width, height].
    
    Returns:
        bool: True if the eye region is valid, False otherwise.
    """
    x, y, width, height = eye_box
    if x < 0 or y < 0 or width <= 0 or height <= 0:
        return False
    if x + width > image.shape[1] or y + height > image.shape[0]:
        return False
    return True


In [87]:
def compare_with_perfect_user(participant_eye_data, perfect_user_eye_data, threshold=0.1):
    """
    Compare the participant's eye data with the perfect user's eye data.
    Args:
        participant_eye_data (list): The eye box and pupil data for the participant.
        perfect_user_eye_data (list): The eye box and pupil data for the perfect user.
        threshold (float): The acceptable threshold for deviation.

    Returns:
        bool: True if the participant data is within the threshold of the perfect user data.
    """
    # Calculate deviation for each data point and check against the threshold
    for p_data, pu_data in zip(participant_eye_data, perfect_user_eye_data):
        if abs(p_data - pu_data) > threshold * abs(pu_data):
            return False
    return True


In [88]:
def extract_key_from_filename(filename):
    # This is where you extract the matching part of the filename.
    # It's an example, you'll need to define how to extract the matching key.
    return filename.split('_')[1]


In [89]:
import os

def process_participant_data(participant_data_path, perfect_user_data, participant_image_folder):
    with open(participant_data_path, 'r') as f:
        for line in f:
            parts = line.strip().split(',')
            if len(parts) < 15:
                print("Invalid data format.")
                continue

            image_name = parts[0]
            image_path = os.path.join(participant_image_folder, image_name)
            key = extract_key_from_filename(image_name)
            perfect_data = perfect_user_data.get(key)

            if not perfect_data:
                print(f"No matching perfect user data for image: {image_name}")
                continue

            # Assuming the first part is the image filename
            participant_eye_box_pupil_data = list(map(float, parts[3:15]))

            # Load and check image
            if not check_image_quality(image_path):
                print(f"Image failed quality check: {image_path}")
                continue

            # Extract eye box data
            left_eye_box = list(map(int, parts[5:9]))
            right_eye_box = list(map(int, parts[11:15]))

            # Apply eye region check
            image = cv2.imread(image_path)
            if image is None or not check_eye_region(image, left_eye_box) or not check_eye_region(image, right_eye_box):
                print(f"Eye region check failed for image: {image_path}")
                continue

            # Compare with perfect user data
            if not compare_with_perfect_user(participant_eye_box_pupil_data, perfect_data, threshold=0.1):
                print(f"Data deviates from 'perfect user' for image: {image_path}")
                continue

            output_path = os.path.join(participant_image_folder, f"processed_{os.path.basename(image_path)}")
            # Save processed data or image
            # This part of the code depends on what "processed" means in your context
            # You need to define how you want to save the processed data or image
            print(f"Data is good and processed image saved to {output_path}")



In [90]:
def load_user_data(user_data_path):
    user_data = {}

    try:
        with open(user_data_path, 'r') as f:
            for line in f:
                parts = line.strip().split(',')
                if len(parts) < 15:
                    continue

                # Assuming the first part is the image filename
                image_name = parts[0]
                eye_box_pupil_data = list(map(float, parts[3:15]))
                
                # Use a part of the image filename as the key, for example, the timestamp
                key = extract_key_from_filename(image_name)
                user_data[key] = eye_box_pupil_data
    except Exception as e:
        print(f"Error loading user data: {e}")

    return user_data


In [97]:

# Function to process the entire dataset
def process_dataset(csv_file, image_folder, perfect_user_data):
    with open(csv_file, 'r') as file:
        for line in file:
            process_participant_data(line.strip().split(',')[0], perfect_user_data, image_folder)  # strip the line to remove newline characters

# Load perfect user data
perfect_user_data = load_user_data('data/William/data.csv') 
print(perfect_user_data)

# Now process the participant's dataset
image_folder = './data/Hossein/images/'
csv_file = './data/Hossein/data.csv'
process_dataset(csv_file, image_folder, perfect_user_data)


{'3b89e44e-f63a-4327-8f7f-2ac6b078cb52.png': [262.0, 199.0, 255.0, 196.0, 22.0, 8.0, 323.0, 203.0, 312.0, 200.0, 22.0, 8.0], '642e7551-3240-4add-b7b4-fd240704519c.png': [261.0, 205.0, 250.0, 203.0, 23.0, 8.0, 323.0, 208.0, 309.0, 205.0, 22.0, 8.0], 'a1e7f7f7-0a0b-4d4e-86b1-a64ec5139ff4.png': [262.0, 209.0, 252.0, 205.0, 23.0, 9.0, 322.0, 210.0, 311.0, 207.0, 22.0, 9.0], '084ef3e3-bffb-48d5-adf6-4e074c13bd7b.png': [261.0, 207.0, 251.0, 205.0, 23.0, 8.0, 320.0, 212.0, 310.0, 208.0, 22.0, 9.0], '92344c06-ea60-44ed-aec1-9f26ec9985a6.png': [261.0, 213.0, 250.0, 209.0, 23.0, 9.0, 324.0, 214.0, 309.0, 211.0, 24.0, 8.0], '1102f111-6bda-4325-99c0-0653ff379be7.png': [253.0, 213.0, 243.0, 210.0, 24.0, 9.0, 315.0, 214.0, 305.0, 212.0, 23.0, 8.0], 'd6ecbcc0-dd9b-4f2b-8304-9b16fd7df2cf.png': [228.0, 201.0, 219.0, 199.0, 21.0, 7.0, 290.0, 204.0, 275.0, 201.0, 24.0, 8.0], '08daa18b-ac91-4c85-967c-364cd702ce76.png': [227.0, 204.0, 218.0, 201.0, 23.0, 8.0, 287.0, 207.0, 277.0, 204.0, 24.0, 8.0], '0dd631

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x89 in position 0: invalid start byte